In [33]:
import pymysql as mysql
import pymysql.cursors
import csv
import pandas as pd
import sqlalchemy
import db_config
import networkx as nx
from collections import (defaultdict,Counter)




In [66]:
#######                                                            #######
#                                                                        #     
#               TRANSLATES GENE SYMBOLS TO ENTREZ AND VV                 #
#                                                                        #  
#######                                                            #######
def genent2sym():

    db = mysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """   SELECT
                    Approved_Symbol,
                    Entrez_Gene_ID_NCBI 
                FROM GenesGO.hgnc_complete
                WHERE Entrez_Gene_ID_NCBI != ''
          """ 

    cursor.execute(sql)
    data = cursor.fetchall()    
#     try: 
#         # execute SQL query using execute() method.
#         cursor.execute(sql)
#         data = cursor.fetchall()
#     except:
#         print('SQL error')
    db.close()

#     t0 = time.time()
    d_sym_ent = {}
    d_ent_sym = {}

    for x in data:
        sym = x[0]
        ent = x[1]
        d_sym_ent[sym] = ent
        d_ent_sym[ent] = sym
#     print(time.time()-t0)
    
    return d_ent_sym, d_sym_ent


#########################################################################
#
#               G E N E S  2  N A M E S
#
#          GENERATES DICT FOR GENE ID 2 NAMES ASSOCIATIONS
#
#########################################################################
def loadgenenames():      
    
    db = mysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")
    cursor = db.cursor()
    # Query to contruct GO Hierarchy
    #
    #          Returns complete list of GO IDs that have a 'is a' relation 
    #
    sql =   """
            SELECT
                Entrez_Gene_ID_NCBI,
                Approved_Symbol,
                Approved_Name
            FROM hgnc_complete
            WHERE GeneStatus LIKE 'Approved'
            """ 
    try: 
        # execute SQL query using execute() method.
#         cursor.Base.execute_query(sql)
        cursor.execute(sql)
        data = cursor.fetchall()
    except:
        print('SQL error')
    db.close()

    d_entz_names = {}
    c = 0
    for xx in data:
        c +=1
        entz = xx[0]
        name = xx[2]
        d_entz_names[entz] = name
        
    return d_entz_names

#########################################################################
#
#               D I S E A S E  2  N A M E S
#
#          GENERATES DICT FOR DISEASE ID 2 NAMES ASSOCIATIONS
#
#########################################################################
def loaddiseasenames():      
    
    db = mysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","Gene2Disease")
    cursor = db.cursor()
    # Query to contruct GO Hierarchy
    #
    #          Returns complete list of GO IDs that have a 'is a' relation 
    #
    sql =  """
            SELECT DISTINCT
                do_id,
                do_name
            FROM disease_ontology
            """ 
    try: 
        # execute SQL query using execute() method.
        cursor.execute(sql)
        data_gd = cursor.fetchall()
    except:
        print('SQL error')
    db.close()

    d_do_names = {}
    c = 0
    for xx in data_gd:
        c +=1
        doid = xx[0]
        name = xx[1]
        d_do_names[doid] = name
        
    return d_do_names



#########################################################################
#
#               G E N E  2  D I S E A S E
#
#          GENERATES DICT FOR GENE TO DISEASE ASSOCIATIONS (PLAIN)
#
#########################################################################
def loadgene2do_plain():      
    
    db = mysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","Gene2Disease")
    cursor = db.cursor()
    # Query to contruct GO Hierarchy
    #
    #          Returns complete list of GO IDs that have a 'is a' relation 
    #
    sql =  """
                SELECT DISTINCT
                    mp.identifier,
                    gd.entrezId
                FROM Gene2Disease.gene2disease gd
                INNER JOIN HumanPhenotypes.umlsmapping mp
                ON mp.disease_ID = gd.diseaseID
                WHERE mp.vocabulary = 'DO'
                """ 
    try: 
        # execute SQL query using execute() method.
        cursor.execute(sql)
        data_gd = cursor.fetchall()
    except:
        print('GO tree SQL error')
    db.close()

    d_gene_do = defaultdict(list)
    c = 0
    for xx in data_gd:
        c +=1
        gene = xx[1]
        do = 'DOID:' + str(xx[0]) 
        d_gene_do[gene].append(do)


    return d_gene_do


#########################################################################
#
#               G E N E  2  G O - F U N C T I O N S
#
#          GENERATES DICT FOR GENE TO GO ASSOCIATIONS (UPSTREAM)
#
#########################################################################
def gene2go(branch_category):
    
    
    if branch_category == 'cell':
        branch = 'C'
    if branch_category == 'mol':
        branch = 'F'
    if branch_category == 'bio':
        branch = 'P'        
    
    db = mysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """ SELECT DISTINCT
                g.entrezid,
                -- g.go_id,
                u.up_ids,
                g.aspect
            FROM GenesGO.Gene2GO_human g
            INNER JOIN GenesGO.GO_upterms u
            on u.go_id = g.go_id
            WHERE g.entrezid != '-'
            AND g.evidence != 'ND' 
            AND g.evidence != 'IEA'
            AND g.evidence != 'IPI'
            AND g.aspect = '%s'
          """ %branch
    try: 
        # execute SQL query using execute() method.
        cursor.execute(sql)
        data = cursor.fetchall()
    except:
        print('SQL error')
    db.close()

#     t0 = time.time()
    d_go_gene = defaultdict(list)
    d_gene_go = defaultdict(list)

    for x in data:
        gene = x[0]
        go = x[1]
        d_gene_go[gene].append(go)
        d_go_gene[go].append(gene)
#     print(time.time()-t0)
    
    return d_gene_go, d_go_gene


#########################################################################
#
#               G E N E  2  G O - F U N C T I O N S
#
#          GENERATES DICT FOR GENE TO GO ASSOCIATIONS (PLAIN)
#
#########################################################################
def gene2go_plain(branch_category):
    
    
    if branch_category == 'cell':
        branch = 'C'
    if branch_category == 'mol':
        branch = 'F'
    if branch_category == 'bio':
        branch = 'P'        
    
    db = mysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")    

    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """ SELECT DISTINCT
                g.entrezid,
                g.go_id,
                g.aspect
            FROM GenesGO.Gene2GO_human g
            WHERE g.entrezid != '-'
            AND g.evidence != 'ND' 
            AND g.evidence != 'IEA'
            AND g.evidence != 'IPI'
            AND g.aspect = '%s'
          """ %branch
    try: 
        # execute SQL query using execute() method.
        cursor.execute(sql)
        data = cursor.fetchall()
    except:
        print('SQL error')
    db.close()

#     t0 = time.time()
    d_go_gene = defaultdict(list)
    d_gene_go = defaultdict(list)

    for x in data:
        gene = x[0]
        go = x[1]
        d_gene_go[gene].append(go)
        d_go_gene[go].append(gene)
#     print(time.time()-t0)
    
    return d_gene_go, d_go_gene

"""

function gets list of GO term-IDs as input 
and returns a dictionary of keys: GO-IDs values: GO-names

"""

def GOnames(category = 'cell'):
    
    if category == 'bio':
        branch = 'biological_process'
    elif category == 'mol':
        branch = 'molecular_function'
    else:
        branch = 'cellular_component'
    
    # Open database connection
    db = mysql.connect("menchelabdb.int.cemm.at","readonly","ra4Roh7ohdee","GenesGO")
    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql_goname = """ SELECT go_id,go_name FROM GO_tree WHERE namespace = '%s' """ %branch 

    try: 
        # execute SQL query using execute() method.
        cursor.execute(sql_goname)
        data_goidname = cursor.fetchall()

    except:
        print('load GO IDs to GO names -  SQL error')
    db.close()
    #print(data_goidname)
    d_go_id_name = {}    
    for x in data_goidname:
        d_go_id_name[x[0]] = (x[1])
    
    return d_go_id_name

#########################################################################
#
#               L O O K U P  
#
#          FOR INTERNAL DB NODE IDS TO ENTREZ
#
#
#########################################################################
def jenids2entzids():
    
    # Open database connection
    db = mysql.connect("asimov.westeurope.cloudapp.azure.com","readonly","ra4Roh7ohdee","ppi")
    # prepare a cursor object using cursor() method
    cursor = db.cursor()

    sql = """
            SELECT 
            id as j_id,
            external_id as entz_id
            FROM ppi.nodes
        """
    try: 
        # execute SQL query 
        cursor.execute(sql)
        data = cursor.fetchall()

    except:
        print('SQL error')

    db.close()    
    
    d_j_e = {}
    d_e_j = {}
 

    c = 0
    for xx in data:
        c +=1
        jid = xx[0]
        eid = xx[1]
        d_j_e[jid] = eid
        d_e_j[eid] = jid


    return d_j_e, d_e_j



In [67]:
d_j_e, d_e_j = jenids2entzids()
d_entz_names = loadgenenames()
d_ent_sym, d_sym_ent = genent2sym()

# disease
d_gene_do = loadgene2do_plain()
d_do_names = loaddiseasenames()

# Functions

d_gene_go, d_go_gene =  gene2go_plain('mol')
d_go_id_name = GOnames(category = 'mol')


In [79]:
db_namespace = 'ppi'
dbconf = db_config.asimov_admin
db = mysql.connect(dbconf["host"], dbconf["user"],
                         dbconf["password"], db='ppi')

In [42]:

cursor = db.cursor()
query = """
        SELECT edges.node1_id, edges.node2_id
        FROM %s.edges
""" % db_namespace

cursor.execute(query)
edges = cursor.fetchall()
G = nx.Graph()

# print(edges)

for x in edges:
    s = x[0]
    t = x[1]
    G.add_edge(s,t)
    
Glcc = G.subgraph(max(nx.connected_components(G), key=len))  # extract lcc graph



In [43]:
print(Glcc.number_of_nodes())
print(Glcc.number_of_edges())

16376
309355


In [ ]:
# generate table with nodeID, degree, namespace 

In [71]:
for node in sorted(Glcc.nodes())[:3]:
#     print(node)
    entz = str((d_j_e[node]))
    print(entz)
    funcs = d_gene_go[entz]
    for f_id in funcs:
        print(d_go_id_name[f_id])
#     print(funcs)
    disease = d_gene_do[entz]
    for d_id in disease:
        print(d_do_names[d_id])
#     print(disease)

41
ligand-gated sodium channel activity
acid-sensing ion channel activity
brain ischemia
5999
calmodulin binding
alcohol dependence
alcohol abuse
bipolar disorder
endometriosis of uterus
endometriosis
schizophrenia
8490
GTPase activator activity
hypertension
essential hypertension
schizophrenia


In [84]:
f = open('ppi_card_lookup.csv','w')

c = 1 
for node in sorted(Glcc.nodes()):
#     print(node)
    entz = str((d_j_e[node]))
#     print(entz)
    symbol = d_ent_sym[entz]
    name = d_entz_names[entz].replace(',',';')
    k = G.degree(node)
    
    funcs = d_gene_go[entz]
    func_str = ''
    fn = 0
    for f_id in funcs:
        if fn < 20:
            try:
                func_str  += d_go_id_name[f_id].replace(',',';') + '|'
                fn += 1
            except:
                pass
        else:
            break
            
    func_str = func_str[:-1]
        
        
    disease = d_gene_do[entz]
    disease_str = ''
    dn = 0
    for d_id in disease:
        if dn < 20:
            disease_str  += d_do_names[d_id].replace(',',';') + '|'
            dn += 1
        else:
            break
    disease_str = disease_str[:-1]
        
        
    
    f.write('%s,%s,%s,%s,%s,%s,%s,%s,%s\n' %(c,node,entz,symbol,name,k,func_str,disease_str,db_namespace))
    c += 1
f.close()

In [77]:
# d_entz_names

In [ ]:
# create and populate table

In [83]:
cursor = db.cursor()


query = "drop table if exists gene_card"
cursor.execute(query)
query = "create table gene_card \
            (id INT NOT NULL,\
             node_id INT,\
             entrezID INT,\
             symbol varchar(15),\
             gene_name varchar(200),\
             degree INT,\
             functions varchar(5000),\
             diseases varchar(5000),\
             namespace varchar(15),\
             PRIMARY KEY (id))"
cursor.execute(query)

# query = """
# INSERT INTO tmp_ars values %s
# """ % ",".join(['("%s", "%s", "%s", "%s")' % (x[0], x[1], x[2], x[3]) for x in relations[['OMIM_id', 'OMIM disease name', 'HPO', 'evidence']].values])
# cursor.execute(query)
# db.commit()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1051, "Unknown table 'ppi.gene_card'")
  result = self._query(query)


0

In [21]:
cursor = db.cursor()

query = """
LOAD DATA LOCAL INFILE 'ppi_degrees_lookup.csv'
INTO TABLE node_degrees
FIELDS TERMINATED BY ','
"""
cursor.execute(query)
db.commit()

InternalError: (1148, 'The used command is not allowed with this MySQL version')

In [28]:
file = '/users/eiofinova/Downloads/omim2phenotype_parsed4dataserver_withname.txt'

relations = pd.read_csv(file, sep='|')
relations

,OMIM disease ID,OMIM disease name,HPO,HPO name,evidence
0,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0000007,Autosomal recessive inheritance,IEA
1,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0003355,Aminoaciduria,IEA
2,OMIM:163600,nipples inverted,HP:0000006,Autosomal dominant inheritance,IEA
3,OMIM:163600,nipples inverted,HP:0003186,Inverted nipples,IEA
4,OMIM:615763,"#615763 cortical dysplasia, complex, with othe...",HP:0002365,Hypoplasia of the brainstem,TAS
...,...,...,...,...,...
95679,OMIM:607475,bothnia retinal dystrophy,HP:0000608,Macular degeneration,IEA
95680,OMIM:607475,bothnia retinal dystrophy,HP:0000662,Nyctalopia,TAS
95681,OMIM:607475,bothnia retinal dystrophy,HP:0000556,Retinal dystrophy,TAS
95682,OMIM:243400,"acetylation, slow",HP:0000007,Autosomal recessive inheritance,IEA


In [29]:
relations.evidence.unique()

array(['IEA', 'TAS', 'PCS'], dtype=object)

In [57]:
# Some of the OMIM IDs were combined with other IDs, so clean that up.
def find_omim(string):
    parts = string.split(';')
    if len(parts) <= 1:
        return string
    for part in parts:
        if part.startswith("OMIM"):
            return part
    return ""

find_omim('PMID:10080184;OMIM:185800')
relations['OMIM_id'] = relations['OMIM disease ID'].apply(find_omim)

In [58]:
relations

,OMIM disease ID,OMIM disease name,HPO,HPO name,evidence,OMIM_id
0,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0000007,Autosomal recessive inheritance,IEA,OMIM:210100
1,OMIM:210100,"beta-aminoisobutyric acid, urinary excretion of",HP:0003355,Aminoaciduria,IEA,OMIM:210100
2,OMIM:163600,nipples inverted,HP:0000006,Autosomal dominant inheritance,IEA,OMIM:163600
3,OMIM:163600,nipples inverted,HP:0003186,Inverted nipples,IEA,OMIM:163600
4,OMIM:615763,"#615763 cortical dysplasia, complex, with othe...",HP:0002365,Hypoplasia of the brainstem,TAS,OMIM:615763
...,...,...,...,...,...,...
95679,OMIM:607475,bothnia retinal dystrophy,HP:0000608,Macular degeneration,IEA,OMIM:607475
95680,OMIM:607475,bothnia retinal dystrophy,HP:0000662,Nyctalopia,TAS,OMIM:607475
95681,OMIM:607475,bothnia retinal dystrophy,HP:0000556,Retinal dystrophy,TAS,OMIM:607475
95682,OMIM:243400,"acetylation, slow",HP:0000007,Autosomal recessive inheritance,IEA,OMIM:243400


In [76]:
db.close()

In [85]:
import db_config
dbconf = db_config.asimov_admin
db = pymysql.connect(dbconf["host"], dbconf["user"],
                         dbconf["password"], db='ppi')
cursor = db.cursor()

In [70]:
query = "drop table if exists tmp_ars"
cursor.execute(query)
query = "create table tmp_ars (OMIM_id varchar(50), OMIM_name varchar(1000), HPO_id varchar(50), comment varchar(5))"
cursor.execute(query)
query = """
INSERT INTO tmp_ars values %s
""" % ",".join(['("%s", "%s", "%s", "%s")' % (x[0], x[1], x[2], x[3]) for x in relations[['OMIM_id', 'OMIM disease name', 'HPO', 'evidence']].values])
cursor.execute(query)
db.commit()

In [67]:
query = "insert into attributes (namespace, external_id, name) select 'OMIM_DISEASE', OMIM_id, max(OMIM_name) from tmp_ars group by 2"
cursor.execute(query)
db.commit()

In [68]:
query = '''
insert into attribute_taxonomies (parent_id, child_id, distance, namespace)
select id, id, 0, namespace from attributes where namespace = 'OMIM_DISEASE'
'''
cursor.execute(query)
db.commit()

In [74]:
query = '''
insert into attribute_relations(attr1_id, attr2_id, comment)
SELECT omims.id, hpos.id, comment
FROM attributes omims
JOIN tmp_ars on omims.external_id = tmp_ars.OMIM_id
JOIN attributes hpos ON hpos.external_id = tmp_ars.HPO_id
'''
cursor.execute(query)
db.commit()

In [75]:
# Same thing, but with attr1 and attr2 switched, for symmetry.
query = '''
insert into attribute_relations(attr2_id, attr1_id, comment)
SELECT omims.id, hpos.id, comment
FROM attributes omims
JOIN tmp_ars on omims.external_id = tmp_ars.OMIM_id
JOIN attributes hpos ON hpos.external_id = tmp_ars.HPO_id
'''
cursor.execute(query)
db.commit()

In [84]:
file = '/users/eiofinova/Downloads/omim2gene_parsed4dataserver.txt'

omimgene = pd.read_csv(file, sep='|', dtype='object')
print(omimgene.columns)
omimgene["OMIM_id"] = "OMIM:" + omimgene["OMIM disease ID"]
omimgene

Index(['OMIM disease ID', 'disease name', 'gene symbol', 'entrezID'], dtype='object')


,OMIM disease ID,disease name,gene symbol,entrezID,OMIM_id
0,202110,17-alpha-hydroxylase/1720-lyase deficiency,CYP17A1,1586,OMIM:202110
1,300438,17-beta-hydroxysteroid dehydrogenase X deficiency,HSD17B10,3028,OMIM:300438
2,204750,2-aminoadipic 2-oxoadipic aciduria,DHTKD1,55526,OMIM:204750
3,610006,2-methylbutyrylglycinuria,ACADSB,36,OMIM:610006
4,273750,3-M syndrome 1,CUL7,9820,OMIM:273750
...,...,...,...,...,...
4247,612956,{Ventricular fibrillation paroxysmal familial 2},DPP6,1804,OMIM:612956
4248,612542,{Vitamin B12 plasma level QTL1},FUT2,2524,OMIM:612542
4249,606579,{Vitiligo-associated multiple autoimmune disea...,NLRP1,22861,OMIM:606579
4250,610379,{West nile virus susceptibility to},CCR5,1234,OMIM:610379


In [88]:
query = "drop table if exists tmp_og"
cursor.execute(query)
query = "create table tmp_og (OMIM_id varchar(50), entrez_id varchar(50))"
cursor.execute(query)
query = """
INSERT INTO tmp_og values %s
""" % ",".join(['("%s", "%s")' % (x[0], x[1]) for x in omimgene[['OMIM_id', 'entrezID']].values])
cursor.execute(query)
db.commit()

In [91]:
query = '''
INSERT INTO nodes_attributes (node_id, attribute_id)
SELECT DISTINCT nodes.id, attributes.id
FROM nodes JOIN tmp_og on nodes.external_id = tmp_og.entrez_id
JOIN attributes ON attributes.external_id = tmp_og.OMIM_id
'''
cursor.execute(query)
db.commit()

In [92]:
query = 'DROP TABLE tmp_ars'
cursor.execute(query)
query = 'DROP TABLE tmp_og'
cursor.execute(query)
db.commit()